# Download Tagesschau Archiv

Tagesschau-Archiv:
- Die letzten Zwölf Monate (incl. aktuellem Monat) werden tagesspezifisch geliefert (typ. eine Seite)
- Davor werden die Monate kommuliert geliefert (paginiert)

Ziel: Keinen Crawler zu schreiben sondern die URLs der Übersichtsseiten generieren und die direkten Links herunterladen

In [1]:
from datetime import date, timedelta
import requests
from bs4 import BeautifulSoup
import json

In [2]:
print(date.today() + timedelta(-1)) 

2025-03-20


In [3]:
today = date.today()
n = -10
urls = []
for i in range(0,n,-1):
    d = today + timedelta(i)
    url = d.strftime("https://www.tagesschau.de/archiv?datum=%Y-%m-%d")
    print(url)
    urls.append(url)

https://www.tagesschau.de/archiv?datum=2025-03-21
https://www.tagesschau.de/archiv?datum=2025-03-20
https://www.tagesschau.de/archiv?datum=2025-03-19
https://www.tagesschau.de/archiv?datum=2025-03-18
https://www.tagesschau.de/archiv?datum=2025-03-17
https://www.tagesschau.de/archiv?datum=2025-03-16
https://www.tagesschau.de/archiv?datum=2025-03-15
https://www.tagesschau.de/archiv?datum=2025-03-14
https://www.tagesschau.de/archiv?datum=2025-03-13
https://www.tagesschau.de/archiv?datum=2025-03-12


In [4]:
article_urls = []
for url in urls:
    print("====>", url)
    page = requests.get(url)
    if page.status_code != 200:
        print("Alert " + url)
        continue
    soup = BeautifulSoup(page.content, 'html.parser')
    articles = soup.find_all("a",{"class" : "teaser-right__link"})
    for article in filter(lambda a: a["href"][0]== "/", articles):
        article_urls.append("https://www.tagesschau.de" + article["href"])
        print(article_urls[-1])

====> https://www.tagesschau.de/archiv?datum=2025-03-21
https://www.tagesschau.de/ausland/europa/tuchel-england-nationaltrainer-104.html
https://www.tagesschau.de/eilmeldung/finanzpaket-bundesrat-102.html
https://www.tagesschau.de/wetter/europa-welt
https://www.tagesschau.de/wetter/deutschland/wettervorhersage-deutschland-100.html
https://www.tagesschau.de/newsticker/liveblog-regierungsbildung-freitag-104.html
https://www.tagesschau.de/wirtschaft/unternehmen/tesla-musk-gruenheide-cybertruck-inzahlung-100.html
https://www.tagesschau.de/ausland/europa/london-heathrow-geschlossen-stromausfall-100.html
https://www.tagesschau.de/newsticker/liveblog-ukraine-freitag-506.html
https://www.tagesschau.de/wirtschaft/verbraucher/klimabilanz-reparatur-autotuer-100.html
https://www.tagesschau.de/ausland/asien/israel-geheimdienst-entlassung-100.html
https://www.tagesschau.de/wirtschaft/biolandwirtschaft-krise-keine-nachfolger-100.html
https://www.tagesschau.de/inland/tsvorzwanzigjahren-ts-142.html
htt

In [5]:
page = requests.get(article_urls[0])
if page.status_code != 200:
    print("Alert " + url)
soup = BeautifulSoup(page.content, 'html.parser')

Variante 1: Die Seite enthält die gewünschten information als application/ld+json-Skript

In [6]:
linked_data = [json.loads(ld.text) for ld in soup.find_all("script",{"type" : "application/ld+json"})]
articlebody = [d["articleBody"] for d in linked_data if "articleBody" in d][0]
print(articlebody)

Heute steht Thomas Tuchel erstmals als Trainer des englischen Nationalteams an der Seitenlinie. Die Royals und den Bolzplatz hat er schonmal auf seiner Seite. Jetzt braucht es nur noch den WM-Titel. Von Mareike Aden Es ist eine große Woche für Thomas Tuchel: Er benannte seinen ersten Kader, gab eine lange Pressekonferenz im Wembley-Stadion, reiste Richtung Norden in die Nähe von Birmingham für das erste Trainingscamp mit seiner neuen Mannschaft im St. George’s Park und stellte sich dem ersten Training vor Journalisten.  Mitte der Woche ging es schon wieder Richtung London. Auf dem weiten makellosen Trainingsgelände der Tottenham Hotspurs war noch eine Pressekonferenz angesetzt, wieder ein offenes Training.  Dabei waren die meisten Kameras auf ihn gerichtet: den Deutschen, der für England den ersten Titel seit 1966 holen soll. Damals ging es gegen Fußball-Erzfeind Deutschland. Seitdem blieben die Three Lions glücklos.  "Die Woche war ein Crashkurs" Tuchel erzählt, er sei ein wenig nervö

Variante 2: Extrahieren der Daten aus HTML: Alle sinnvollen p-Tags im article-Tag

In [7]:
article = soup.find("article")
print(article)

<article class="container content-wrapper__group">
<div class="seitenkopf">
<div class="seitenkopf__media columns twelve m-twelve">
<div class="ts-picture__wrapper">
<picture class="ts-picture ts-picture--topbanner">
<source media="(max-width: 420px)" srcset="https://images.tagesschau.de/image/5347905a-21c2-43d9-a7bf-1f6aaac4dd1b/AAABlbhFixs/AAABkZLkm68/16x9-640/thomas-tuchel-472.webp" type="image/webp"/>
<source media="(max-width: 767px)" srcset="https://images.tagesschau.de/image/5347905a-21c2-43d9-a7bf-1f6aaac4dd1b/AAABlbhFixs/AAABkZLk4K0/16x9-768/thomas-tuchel-472.webp" type="image/webp"/>
<source media="(max-width:  1023px)" srcset="https://images.tagesschau.de/image/5347905a-21c2-43d9-a7bf-1f6aaac4dd1b/AAABlbhFixs/AAABkZLp4nc/20x9-960/thomas-tuchel-472.webp" type="image/webp"/>
<source media="(min-width: 1024px)" srcset="https://images.tagesschau.de/image/5347905a-21c2-43d9-a7bf-1f6aaac4dd1b/AAABlbhFixs/AAABkZLpihI/20x9-1280/thomas-tuchel-472.webp" type="image/webp"/>
<source med

In [8]:
paragraphs = article.findAll(lambda tag: tag.name=="p" and tag.has_attr("class") and tag.attrs["class"][0].startswith("textabsatz"))
for p in paragraphs:
    print(p.text)


Heute steht Thomas Tuchel erstmals als Trainer des englischen Nationalteams an der Seitenlinie. Die Royals und den Bolzplatz hat er schonmal auf seiner Seite. Jetzt braucht es nur noch den WM-Titel.


Es ist eine große Woche für Thomas Tuchel: Er benannte seinen ersten Kader, gab eine lange Pressekonferenz im Wembley-Stadion, reiste Richtung Norden in die Nähe von Birmingham für das erste Trainingscamp mit seiner neuen Mannschaft im St. George’s Park und stellte sich dem ersten Training vor Journalisten.

Mitte der Woche ging es schon wieder Richtung London. Auf dem weiten makellosen Trainingsgelände der Tottenham Hotspurs war noch eine Pressekonferenz angesetzt, wieder ein offenes Training.

Dabei waren die meisten Kameras auf ihn gerichtet: den Deutschen, der für England den ersten Titel seit 1966 holen soll. Damals ging es gegen Fußball-Erzfeind Deutschland. Seitdem blieben die Three Lions glücklos.

Tuchel erzählt, er sei ein wenig nervös, wie immer vor Spielen. Aber er wirkt gut 

C:\Users\paulf\AppData\Local\Temp\ipykernel_3108\1891106229.py:1: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  paragraphs = article.findAll(lambda tag: tag.name=="p" and tag.has_attr("class") and tag.attrs["class"][0].startswith("textabsatz"))


url = "https://www.tagesschau.de/archiv?datum=2023-01-01"
page = requests.get(url)
if page.status_code != 200:
    alert("Geht nicht!")

In [9]:
url = "https://www.tagesschau.de/archiv?datum=2023-01-01"
page = requests.get(url)
if page.status_code != 200:
    alert("Geht nicht!")
soup = BeautifulSoup(page.content, 'html.parser')
pages = soup.findAll(lambda tag: tag.name=="a" and tag.has_attr("class") and tag.attrs["class"][0] == "paginierung__liste--link")
for p in pages:
    print(p.attrs["href"], "'" + p.text + "'")

?datum=2023-01-01&pageIndex=2 '2'
?datum=2023-01-01&pageIndex=3 '3'
?datum=2023-01-01&pageIndex=8 '8'
?datum=2023-01-01&pageIndex=2 '
Pfeil rechts




'


C:\Users\paulf\AppData\Local\Temp\ipykernel_3108\14890863.py:6: DeprecationWarning: Call to deprecated method findAll. (Replaced by find_all) -- Deprecated since version 4.0.0.
  pages = soup.findAll(lambda tag: tag.name=="a" and tag.has_attr("class") and tag.attrs["class"][0] == "paginierung__liste--link")
